## Overview
This notebook loads a collection of synthetic FHIR bundles and value sets and shows some simple queries. Running this first will set up the environment for other notebooks in the tutorial

## Setup Tasks
Some setup before the real show begins...

In [1]:
from pyspark.sql import SparkSession
import time

# Enable Hive support for our session so we can save resources as Hive tables
spark = SparkSession.builder \
                    .config('hive.exec.dynamic.partition.mode', 'nonstrict') \
                    .enableHiveSupport() \
                    .getOrCreate()

## Import Synthetic Data
This tutorial uses data generated by Synthea. It is simply a directory of STU3 bundles visible included in the tutorial; you can see it in the bundles directory.

Let's load the bundles and examine a couple data types in them.

In [2]:
from bunsen.stu3.bundles import load_from_directory, extract_entry, write_to_database

# Load and cache the bundles so we don't reload them every time.
bundles = load_from_directory(spark, 'gs://healthedatalab/synthea/fhir').cache()

# Get the observation and encounters
observations = extract_entry(spark, bundles, 'observation')
encounters = extract_entry(spark, bundles, 'encounter')

## Bunsen documentation
To get help using functions like *load_from_directory* or *extract_entry*, you can see the documentation at https://engineering.cerner.com/bunsen or via Python's help system, like this:

In [3]:
help(extract_entry)

Help on function extract_entry in module bunsen.stu3.bundles:

extract_entry(sparkSession, javaRDD, resourceName)
    Returns a dataset for the given entry type from the bundles.
    
    :param sparkSession: the SparkSession instance
    :param javaRDD: the RDD produced by :func:`load_from_directory` or other methods
        in this package
    :param resourceName: the name of the FHIR resource to extract
        (condition, observation, etc)
    :return: a DataFrame containing the given resource encoded into Spark columns



## Generated from FHIR Resource Definitions
The Apache Spark datasets used here are fully generated from the FHIR resource definitions, with every field mapped one-to-one. For instance, here is the fully Spark schema of the Observation resource:

In [4]:
observations.printSchema()

root
 |-- id: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: s

## Load some data
The next step will load some data and inspect it. Since Spark lazily delays execution until output is needed, all of the work will be done here. This can take several seconds or longer depending on the machine, but users can check its status by looking at the [Spark application page](http://localhost:4040).

For now, let's just turn our encounter resources into a simple table of all encounters since 2013:

In [ ]:
from pyspark.sql.functions import col
start = time.time()

encounters.select('subject.reference', 
                  'class.code', 
                  'period.start', 
                  'period.end') \
          .where(col('start') > '2013') \
          .limit(10) \
          .toPandas()
end = time.time()
print(end - start)

## Exploding nested lists
FHIR's nested structures group related data, making many workloads simpler. We can reference such nested structures directly, and "explode" nested lists when needed to analyze them. Let's build a table of all observation codes in our data:

In [6]:
from pyspark.sql.functions import explode
start = time.time()

codes = observations.select('subject',
                            explode('code.coding').alias('coding')) \
                    .select('subject.reference', 
                            'coding.system', 
                            'coding.code',
                            'coding.display')
end = time.time()
print(end - start)                    
codes.limit(10).toPandas()

,reference,system,code,display
0,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,8302-2,Body Height
1,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,29463-7,Body Weight
2,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,39156-5,Body Mass Index
3,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,55284-4,Blood Pressure
4,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,2093-3,Total Cholesterol
5,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,2571-8,Triglycerides
6,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,18262-6,Low Density Lipoprotein Cholesterol
7,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,2085-9,High Density Lipoprotein Cholesterol
8,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,19926-5,FEV1/​FVC
9,urn:uuid:f88c719e-9875-446d-852b-cddb79fe4998,http://loinc.org,8302-2,Body Height


## Analyzing data
Our datasets become much easier to analyze once they've been projected onto a simpler model that suits the proble at hand. The code below simply shows the most frequent observation codes in our synthetic data.

In [7]:
start = time.time()
codes.groupBy('system', 'code', 'display') \
     .count() \
     .orderBy('count', ascending=False) \
     .limit(10) \
     .toPandas()
end = time.time()
print(end - start) 

,system,code,display,count
0,http://loinc.org,4548-4,Hemoglobin A1c/Hemoglobin.total in Blood,1753
1,http://loinc.org,55284-4,Blood Pressure,1377
2,http://loinc.org,8302-2,Body Height,1377
3,http://loinc.org,29463-7,Body Weight,1377
4,http://loinc.org,39156-5,Body Mass Index,1350
5,http://loinc.org,6299-2,Urea Nitrogen,871
6,http://loinc.org,2069-3,Chloride,871
7,http://loinc.org,38483-4,Creatinine,871
8,http://loinc.org,2339-0,Glucose,871
9,http://loinc.org,6298-4,Potassium,871


## Writing resources to a database
Directly loading JSON or XML FHIR bundles is useful for ingesting and early exploration of data, but a more efficient format works better repeated use. Since Bunsen encodes resources natively in Apache Spark dataframes, we can take advantage of Spark's ability to write it to a Hive database. Bunsen offers the *write_to_database* function as a convenient way to write resources from bundles to a database, with a table for each resource. 

Note that each table preserves the original, nested structure definition of the FHIR resource, and is field-for-field equivalent. 

The cell below will save our test data to tables in the "tutorial_small" database. When running it, you can see progress in the Spark UI at http://localhost:4040.


In [8]:
resources = ['allergyintolerance',
             'careplan',
             'claim',
             'condition',
             'encounter',
             'immunization',
             'medication',
             'medicationrequest',
             'observation',
             'organization',
             'patient',
             'procedure']

start = time.time()
write_to_database(spark, 
                  bundles, 
                  'tutorial_small',
                  resources)
end = time.time()
print(end - start)  

## Reading from a Hive database
Now that we've saved our data to a Hive database, we can easily view and query the tables with Spark SQL:

In [9]:
spark.sql('use tutorial_small')
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,tutorial_small,allergyintolerance,False
1,tutorial_small,careplan,False
2,tutorial_small,claim,False
3,tutorial_small,condition,False
4,tutorial_small,encounter,False
5,tutorial_small,immunization,False
6,tutorial_small,medication,False
7,tutorial_small,medicationrequest,False
8,tutorial_small,observation,False
9,tutorial_small,organization,False


In [10]:
start = time.time()
spark.sql("""
select subject.reference, 
       count(*) cnt
from encounter
where class.code != 'WELLNESS' and
      period.start > '2013'
group by subject.reference
order by cnt desc
limit 10
""").toPandas()
end = time.time()
print(end - start)  

,reference,cnt
0,urn:uuid:e206880c-7762-4aee-a3e2-5a8c89512c18,53
1,urn:uuid:e538491e-cf8e-4a3f-97a5-45811e066f27,44
2,urn:uuid:dcad3c44-64de-43b6-b24c-989f8f27c71d,33
3,urn:uuid:5804a9d3-3518-4862-a1e4-a61b0f1a4be4,31
4,urn:uuid:2bf9eab0-fec0-41b2-9f91-3369e38b98f6,19
5,urn:uuid:90a7ded5-a5ce-43df-b973-7bc7ce7a3011,18
6,urn:uuid:8f538e46-a1d1-4c75-beb7-e3946124e730,16
7,urn:uuid:6f58dbea-7532-4090-97a8-79982bab98f5,12
8,urn:uuid:aa251e83-9a9b-446f-ba2f-87e2da7c4d34,8
9,urn:uuid:73bbd5a3-00b5-4216-bd5d-601359ca9e42,6
